### Get Spotify credentials

In [13]:
# Get required info for token request
import os
from dotenv import load_dotenv
load_dotenv()
client_id = os.getenv("SPOTIFY_CLIENT_ID")
client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")
token_url = "https://accounts.spotify.com/api/token"

# Perform OAuth client credentials authorization
import requests
r = requests.post(
    "https://accounts.spotify.com/api/token",
    data={ "grant_type": "client_credentials" },
    auth=(client_id, client_secret)
)
r.raise_for_status()
response = r.json()
token = response["access_token"]
headers = { "Authorization": f"Bearer {token}" }

### Get Song a Day playlist data

In [18]:
playlists = { 2019: "6xJjyO5AGrZkyGkWYXfUX8", 2020: "3iNn4yKx9PxpSVYfRXkK8i" }
years = sorted(playlists.keys())
tracks = { year: [] for year in years }

# For each year's playlist, grab tracks
for year in years:
    # Page through track list for the playlist
    next_url = f"https://api.spotify.com/v1/playlists/{playlists[year]}/tracks"
    while next_url:
        # Fetch the current page
        r = requests.get(next_url, headers=headers)
        r.raise_for_status()
        response = r.json()
        # Reshape each playlist item and add to our "tracks" list for the relevant year
        for playlist_item in response["items"]:
            tracks[year].append({
                "name": playlist_item["track"]["name"],
                "id": playlist_item["track"]["id"],
                "album_id": playlist_item["track"]["album"]["id"],
                "album_name": playlist_item["track"]["album"]["name"],
                "artists": [
                    { "name": artist["name"], "id": artist["id"] }
                    for artist in playlist_item["track"]["artists"]
                ],
                "primary_artist_name": playlist_item["track"]["artists"][0]["name"],
                "primary_artist_id": playlist_item["track"]["artists"][0]["id"],
                "is_local": playlist_item["is_local"]
            })
        # Take note of the URL of the next page of playlist items
        next_url = response["next"]